In [11]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera
from scipy.signal import find_peaks, butter, filtfilt
from simple_pid import PID
#from ipywidgets import interact, interactive
from pwm import PWM
import ipywidgets as ipw
import time
import matplotlib.pyplot as plt
import skimage as ski
import numpy as np
import time
import threading

# Takes in a percent and a pwm, sets the speed according to the percent
def setEngineSpeed(percent, pwm):
    # If the percent is less than 0, set to 0
    if percent < 0:
        percent = 0
	# If the percent is greater than 1, set to 1
    if percent > 1:
        percent = 1
	# Calculate the speed to set the motor
    newSpeed = int(1000000 * percent + 1000000)
    #print("set engine speed ", speed)
	# Set the duty_cylce to the speed
    pwm.duty_cycle = newSpeed

# Takes in a percent and a pwm, turns to the right according to the percent
def turnRight(percent, pwm):
	# If the percent is less than 0, set to 0
    if percent < 0:
        percent = 0
	# If the percent is greater than 1, set to 1
    if percent > 1:
        percent = 1
	# Calculate the direction to set the servo
    value = int(1500000 - 500000 * percent)
    #print("turn right speed: ", value)
	# Set the duty_cylce to the value
    pwm.duty_cycle = value

# Takes in a percent and a pwm, turns to the left according to the percent
def turnLeft(percent, pwm):
	# If the percent is less than 0, set to 0
    if percent < 0:
        percent = 0
	# If the percent is greater than 1, set to 1
    if percent > 1:
        percent = 1
	# Calculate the direction to set the servo
    value = int(500000 * percent + 1500000)
    #print("turn left speed: ", value)
	# Set the duty_cylce to the value
    pwm.duty_cycle = value

def turn(percent, pwm):
    if percent < -1:
        percent = -1
    if percent > 1:
        percent = 1
    if percent < 0:
        turnRight(-1 * percent, pwm)
    else:
        turnLeft(percent, pwm)
        
def getNewSpeed(turnPercent):
    if turnPercent <= 0.15:
        newSpeed = 0.22
    elif turnPercent <=0.5:
        newSpeed = 0.17
    else:
        newSpeed = 0.16
    return newSpeed

def setInteractValues(speed = 0, p = 1, i = 0, d = 0):
    global Kp 
    Kp = p
    global Ki 
    Ki = i
    global Kd 
    Kd = d
    global engineSpeed
    engineSpeed = speed


In [12]:
speed = ipw.FloatSlider(min=0.0, max=1.0, step=0.01, value=0, description='Speed')
display(speed)
Kp = ipw.FloatSlider(min=0.0, max=10.0, step=0.1, value=0.8, description='Kp')
display(Kp)
Ki = ipw.FloatSlider(min=0.0, max=10.0, step=0.1, value=0.5, description='Ki')
display(Ki)
Kd = ipw.FloatSlider(min=0.0, max=10.0, step=0.05, value=0.8, description='Kd')
display(Kd)
running = ipw.IntSlider(min=0, max=1, step=1, value=1, description='KILL SWITCH')
display(running)

button = ipw.ToggleButton(value=False, description='Snap photo', disabled=False,button_style='', tooltip='Description', icon='check')
display(button)

FloatSlider(value=0.0, description='Speed', max=1.0, step=0.01)

FloatSlider(value=0.8, description='Kp', max=10.0)

FloatSlider(value=0.5, description='Ki', max=10.0)

FloatSlider(value=0.8, description='Kd', max=10.0, step=0.05)

IntSlider(value=1, description='KILL SWITCH', max=1)

ToggleButton(value=False, description='Snap photo', icon='check', tooltip='Description')

In [13]:
# Setup and export pwm0 and pwm1
pwm0 = PWM(0)
pwm1 = PWM(1)
pwm0.export()
pwm1.export()

# Set the periods to 20ms
pwm0.period = 20000000
pwm1.period = 20000000

# Set the duty_cycle of the motor to 1ms and servo to 1.5ms
setEngineSpeed(0, pwm1)
turnRight(0, pwm0)

# Enable the pwms
pwm0.enable = True
pwm1.enable = True

# Give some time to start up
time.sleep(5)

# START OF COMPUTER VISION

res = (640,480)
b, a = butter(3, 0.007)


camera = PiCamera()


# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 60

# Initialize the buffer and start capturing
rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

# Measure the time needed to process 300 images to estimate the FPS
output = 0
engineSpeed = 0
whiteCenterLine = (0,0)
turning = 'n'
images = []

def func():
    global turning
    global images
    frame_num = 6
    for f in stream:
        setEngineSpeed(speed.value, pwm1)
        pid = PID(Kp.value, Ki.value, Kd.value, setpoint=0)
        pid.sample_time = 0.05
        # Get the intensity component of the image (a trick to get black and white images)
        I = f.array[:, :, 0]
        if (button.value):
            frame_num = 0
            button.value = False
        if (frame_num <=2):
            images.append(I)
            frame_num = frame_num + 1
        # Reset the buffer for the next image
        rawCapture.truncate(0)

        # Select a horizontal line in the middle of the image
        ################## Lets play around with this value to see if there is a better spot
        L = I[440, :]

        # Smooth the transitions so we can detect the peaks 
        Lf = filtfilt(b, a, L)

        # Find peaks which are higher than 0.5
        p = find_peaks(Lf, height=128)
        num_peaks = len(p[0])
        #for x in range(num_peaks):
            #p[0][x] -=25;
        if num_peaks == 1:
            whiteCenterLine = p[0][0]
        elif num_peaks >= 2:
            whiteCenterLine = (p[0][0] + p[0][len(p)-1]) / 2
        elif (num_peaks == 0):
            pass # REMEMBER LAST TURN
        # Print peaks
        # print(p)

        # Here we should steer and then update to the next frame.
        #print("num peaks ", num_peaks)
        if num_peaks == 0:
            if turning == 'l':
                output = 1
            elif turning == 'r':
                output = -1
        else:
            if turning == 'l' and whiteCenterLine - 320 > 200:
                pass
            elif turning == 'r' and whiteCenterLine < 200:
                pass
            else:
                percentOff = (whiteCenterLine - 320) / 320.0
                output = pid(percentOff)
                if whiteCenterLine < 320:
                    turning = 'l'
                else:
                    turning = 'r'
        #print("output: ", output)
        newSpeed = getNewSpeed(abs(output))
        # print("speed: ", newSpeed)
        if newSpeed <= speed.value:
            setEngineSpeed(newSpeed, pwm1)
        turn(output, pwm0)

        if not running.value:
            break
    # Stop the motor
    setEngineSpeed(0, pwm1)
    # Reposition the servo to 0
    turnRight(0, pwm0)
    time.sleep(.1)

    #time_elapsed = time.time() - t
    #print("Elapsed {:0.2f} seconds, estimated FPS {:0.2f}".format(time_elapsed, N / time_elapsed))

    stream.close()
    rawCapture.close()
    camera.close()

    # Disable both pwms
    pwm0.enable = False
    pwm1.enable = False
            
t = threading.Thread(target=func)
t.start()

PiCameraMMALError: Failed to enable connection: Out of resources

In [ ]:
stream.close()
rawCapture.close()
camera.close()

In [ ]:
for i in range(0,len(images)): 
    plt.imshow(images[i])

In [ ]:
plt.figure(figsize=(18, 24))

for k, img in enumerate(images):
    ax = plt.subplot(4, 3, k + 1)
    ax.imshow(img)
    
plt.tight_layout()